## DEMO

### CSV

````Enercon````

In [1]:
from models import Enercon_Reports_Processor; from pathlib import Path
Enercon_Reports_Processor(
    input_folder=Path("data") / "enercon" / "input" 
)(
    metadata_output_folder = Path("data") / "enercon" / "output" / "metadatas",
    inspection_checklist_output_folder = Path("data") / "enercon" / "output" / "inspection_checklists"
)

``Vestas``

In [2]:
from models import  Vestas_Reports_Processor; from pathlib import Path
Vestas_Reports_Processor(
    input_folder=Path("data") / "vestas" / "input"
)(
    metadata_output_folder=Path('data') / 'vestas' / 'output' / 'metadatas',
    inspection_checklist_output_folder=Path('data') / 'vestas' / 'output' / 'inspection_checklists'
)

### SQL

````Build database````

In [2]:
from models import DatabaseManager as db; db().build()

````Defining tables````

In [3]:
from models import DatabaseManager as db; from pathlib import Path 
db().create_tables_from_sql(
    Path('database_building')
)

Executed create_service_reports_checklists.sql
Executed create_service_reports_metadas.sql
Executed table_ingestion_tracking.sql


````Insert pdf reports into database````

In [5]:
from pathlib import Path
import pandas as pd

In [9]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text

def categorize_inspection_items(pdf_path: Path) -> pd.DataFrame:
    """
    Add item_category and item_number columns based on the 'no' column content.
    
    Args:
        pdf_path (Path): Path to the CSV file to process
        
    Returns:
        pd.DataFrame: Processed DataFrame with:
            - Two new columns at the start: 'item_category' and 'item_number'
            - Category rows removed
            - Original columns preserved: ['no', 'check_item', 'result']
    """
    # Read CSV without using first column as index
    inspection_checklist = pd.read_csv(pdf_path)
    
    current_category = None
    category_counter = 1  # Start counter at 1 instead of 0
    
    # Create lists to store categories and numbers
    categories = []
    numbers = []
    
    # Iterate through rows
    for idx, row in inspection_checklist.iterrows():
        # Check if it's a category row (no numeric value in 'no' column)
        if not str(row['no']).replace('.', '').isdigit():
            current_category = row['no']  # Category is in 'no' column
            category_counter += 1
            categories.append(None)
            numbers.append(None)
        else:
            categories.append(current_category)
            numbers.append(category_counter - 1)  # Use previous counter value
    
    # Insert new columns at the beginning
    inspection_checklist.insert(0, 'item_number', pd.Series(numbers, dtype='Int64'))
    inspection_checklist.insert(0, 'item_category', categories)
    
    # Drop category rows and unwanted column
    result_df = inspection_checklist[inspection_checklist['no'].str.replace('.', '').str.isdigit()]
    return result_df.drop(columns=['Unnamed: 0'])

def process_inspection_reports(folder_path: Path) -> pd.DataFrame:
    """
    Process all inspection reports in a folder, adding report_id, categories and item numbers.
    
    Args:
        folder_path (Path): Path to the folder containing inspection CSV files
        
    Returns:
        pd.DataFrame: Concatenated DataFrame of all reports with:
            - report_id column added
            - item_category and item_number columns
            - Category rows removed
            - Original columns preserved
    """
    all_reports = []
    report_id = 1
    
    # Get all CSV files in the folder
    for csv_file in folder_path.glob("inspection_*.csv"):
        # Process single report
        df = categorize_inspection_items(csv_file)
        
        # Add report_id
        df.insert(0, 'report_id', report_id)
        
        all_reports.append(df)
        report_id += 1
    
    # Concatenate all reports
    return pd.concat(all_reports, ignore_index=True)

# Process all Enercon reports
enercon_folder = Path("data/enercon/output/inspection_checklists")
all_categorized_reports = process_inspection_reports(enercon_folder)

# Create and populate database
engine = create_engine('sqlite:///test_base.db')
all_categorized_reports.to_sql('inspection_checklists', engine, if_exists='replace', index=False)

# Verify insertion
print("Data inserted into test_base.db")
print("\nVerification query:")
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM inspection_checklists")).fetchone()
    print(f"Total rows in database: {result[0]}")

Data inserted into test_base.db

Verification query:
Total rows in database: 550


In [35]:
def process_inspection_reports(folder_path: Path) -> pd.DataFrame:
    """
    Process all inspection reports in a folder, adding report_id, categories and item numbers.
    
    Args:
        folder_path (Path): Path to the folder containing inspection CSV files
        
    Returns:
        pd.DataFrame: Concatenated DataFrame of all reports with:
            - report_id column added
            - item_category and item_number columns
            - Category rows removed
            - Original columns preserved
    """
    all_reports = []
    report_id = 1
    
    # Get all CSV files in the folder
    for csv_file in folder_path.glob("inspection_*.csv"):
        # Process single report
        df = categorize_inspection_items(csv_file)
        
        # Add report_id
        df.insert(0, 'report_id', report_id)
        
        all_reports.append(df)
        report_id += 1
    
    # Concatenate all reports
    return pd.concat(all_reports, ignore_index=True)

# Test with Enercon reports
enercon_folder = Path("data/enercon/output/inspection_checklists")
all_categorized_reports = process_inspection_reports(enercon_folder)

print("Total shape:", all_categorized_reports.shape)
print("\nFirst few rows:")
display(all_categorized_reports.head())
print("\nUnique report IDs:", all_categorized_reports['report_id'].unique())

Total shape: (550, 6)

First few rows:


,report_id,item_category,item_number,no,check_item,result
0,1,Tower base,1,1,Carrying out a visual inspection of the tower ...,1 Done - No defects
1,1,Tower base,1,2,Checking RCD tower base,1 Done - No defects
2,1,Tower,2,3,Visually inspecting the tower,1 Done - No defects
3,1,Machine house,3,4,Checking the machine house first aid kit,1 Done - No defects
4,1,Machine house,3,5,Checking the fire extinguisher in the machine ...,1 Done - No defects



Unique report IDs: [1 2 3 4]


In [36]:
all_categorized_reports

,report_id,item_category,item_number,no,check_item,result
0,1,Tower base,1,1,Carrying out a visual inspection of the tower ...,1 Done - No defects
1,1,Tower base,1,2,Checking RCD tower base,1 Done - No defects
2,1,Tower,2,3,Visually inspecting the tower,1 Done - No defects
3,1,Machine house,3,4,Checking the machine house first aid kit,1 Done - No defects
4,1,Machine house,3,5,Checking the fire extinguisher in the machine ...,1 Done - No defects
...,...,...,...,...,...,...
545,4,Report overview,26,173,"rotor system - central lubrication, pressure g...","special, grease system - not working"
546,4,Report overview,26,175,"main component, screwing - Hub adapter – blade...",fit/gap size - wrong alignment
547,4,Report overview,26,176,"main component, screwing - rotor hub - blade f...",fit/gap size - wrong alignment
548,4,Report overview,26,177,"main component, screwing - rotor hub - blade f...",fit/gap size - wrong alignment


In [ ]:
from sqlalchemy import create_engine

# Create test database
engine = create_engine('sqlite:///test_base.db')

# Insert the DataFrame into the database
all_categorized_reports.to_sql('inspection_checklists', engine, if_exists='replace', index=False)

# Verify the data
print("Data inserted into test_base.db")
print("\nVerification query:")
with engine.connect() as conn:
    result = conn.execute("SELECT COUNT(*) FROM inspection_checklists").fetchone()
    print(f"Total rows in database: {result[0]}")

In [10]:
def process_metadata_files(folder_path: Path) -> pd.DataFrame:
    """
    Process all metadata files, adding report_id
    """
    all_metadata = []
    report_id = 1
    
    # Get all metadata files
    for metadata_file in folder_path.glob("metadata_*.csv"):
        # Read metadata file
        df = pd.read_csv(metadata_file)
        
        # Add report_id
        df.insert(0, 'report_id', report_id)
        
        all_metadata.append(df)
        report_id += 1
    
    # Concatenate all metadata
    return pd.concat(all_metadata, ignore_index=True)

# Process Enercon metadata
enercon_folder = Path("data/enercon/output/metadata")
all_metadata = process_metadata_files(enercon_folder)

# Add to database
engine = create_engine('sqlite:///test_base.db')
all_metadata.to_sql('metadata', engine, if_exists='replace', index=False)

# Verify insertion
print("Metadata inserted into test_base.db")
print("\nVerification query:")
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM metadata")).fetchone()
    print(f"Total metadata rows in database: {result[0]}")

ValueError: No objects to concatenate

<div style="color: lightcoral ; font-weight: bold">⚠️ Danger zone ⚠️</div>

````Drop tables````

In [ ]:
from models import DatabaseManager as db; db().drop_tables()

````Delete Database````

In [1]:
from models import DatabaseManager as db; db().delete_file()

Database file data/wpd_windmanager_test_database.db successfully deleted
